In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["MKL_DYNAMIC"] = "FALSE"
from threadpoolctl import threadpool_limits, threadpool_info

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
from pathlib import Path
import openpyxl
import itertools
import torch

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, friedmanchisquare, wilcoxon
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

from threadpoolctl import threadpool_limits, threadpool_info


Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Python 3.12 and above currently is not supported by Spark and Ray. Please note that some functionality will not work and currently is not supported.


### Read Datasets

In [2]:
borderline_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline_data.parquet") 
borderline2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline3_data.parquet")
smote_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_data.parquet")
smote2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_data.parquet")
gan_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN_data.parquet")
gan2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN3_data.parquet")

original_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_train.parquet")

In [ ]:
print(datacompy.Compare(original_df, borderline_df, join_columns='Label').report())

In [ ]:
borderline_df.drop_duplicates(inplace=True)

In [ ]:
original_df["Label"].value_counts().sum()

In [ ]:
print(smote_df["Label"].value_counts())

In [ ]:
print(smote_df["Label"].value_counts())
print(borderline_df["Label"].value_counts())

In [3]:
mix_df = pd.concat([borderline_df, borderline2_df, smote_df, smote2_df, gan_df, gan2_df], ignore_index=True)
smote_df = pd.concat([smote_df, smote2_df], ignore_index=True)
gan_df = pd.concat([gan_df, gan2_df], ignore_index=True)
borderline_df = pd.concat([borderline_df, borderline2_df], ignore_index=True)

In [4]:
print(original_df["Label"].value_counts())
count1=original_df["Label"].value_counts().sum()
count2=abs((original_df['Label']==0).sum() - (original_df['Label']==1).sum())
print(f"Before undersampling: {count2}")
print(f"After number of samples: {count1+count2}")

Label
0    2415460
1      62708
Name: count, dtype: int64
Before undersampling: 2352752
After number of samples: 4830920


In [5]:
#Split data
X_mix, y_mix = mix_df.drop(columns=["Label", "source"]), mix_df["Label"]
X_smote, y_smote = smote_df.drop(columns=["Label", "source"]), smote_df["Label"]
X_GAN, y_GAN = gan_df.drop(columns=["Label", "source"]), gan_df["Label"]
X_borderline, y_borderline = borderline_df.drop(columns=["Label", "source"]), borderline_df["Label"]

#Dictionary
data = {}
data["mix"] = (X_mix, y_mix)
data["smote"] = (X_smote, y_smote)
data["GAN"] = (X_GAN, y_GAN)
data["borderline"] = (X_borderline, y_borderline)

compare = {}
compare["mix"] = mix_df
compare["smote"] = smote_df
compare["GAN"] = gan_df
compare["borderline"] = borderline_df

### Undersampling

In [ ]:
base = Path("D:/ml/undersampling_data/data/cicunsw/reduced")
#base2 = Path("D:/ml/undersampling_data/models/kmeans&centroids2")

df_ = {}

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class
    X_majority = X_train[y_train == 1]
    
    KM = KMeans(n_clusters=(int)(count2))
    with threadpool_limits(limits=16):
    # Apply KMeans clustering
        kmeans = KM.fit(X_majority)
    
    # Create a DataFrame for centroids
    X_majority_reduced = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    y_majority_reduced = pd.Series([1] * (int)(count2), name="Label") 
    
    # Combine reduced majority class with original minority class
    df_majority = pd.concat([X_majority_reduced, y_majority_reduced], axis=1).reset_index(drop=True)
    
    # Add source column if not present
    df_majority["source"] = None
    missing_source = df_majority[df_majority["source"].isna()]
    if not missing_source.empty:
        df_majority.loc[df_majority["source"].isna(), "source"] = "centroid" 
    
    print(df_majority)
    df_majority = df_majority.reindex(columns=original_df.columns, fill_value=0.0)
    
    # Combine with original minority class
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True)  
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_centroids.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

In [6]:
def _as_float32_c(X):
    return np.asarray(X, dtype=np.float32, order="C")

def _batch_iter_df(Xdf: pd.DataFrame, chunk_size: int):
    n = len(Xdf)
    for s in range(0, n, chunk_size):
        # bez zbędnych kopii
        yield Xdf.iloc[s:s+chunk_size].to_numpy(dtype=np.float32, copy=False)

def _batch_iter_np(Xnp: np.ndarray, chunk_size: int):
    Xnp = _as_float32_c(Xnp)
    n = Xnp.shape[0]
    for s in range(0, n, chunk_size):
        yield Xnp[s:s+chunk_size]

def _iter_batches(X, chunk_size: int):
    if isinstance(X, pd.DataFrame):
        return _batch_iter_df(X, chunk_size)
    return _batch_iter_np(X, chunk_size)

def _reservoir_sample_rows(X, sample_size: int, chunk_size: int, rng):
    """Próbkowanie rezerwuarowe bez trzymania całości w RAM."""
    buf = None
    seen = 0
    for Xb in _iter_batches(X, chunk_size):
        if buf is None:
            take = min(sample_size, len(Xb))
            buf = Xb[:take].copy(order="C")
            seen = take
            # jeśli cały batch mniejszy niż sample -> dokładamy
            if take < sample_size and len(Xb) > take:
                extra = Xb[take: min(sample_size, len(Xb))].copy(order="C")
                buf = np.vstack([buf, extra])
                seen = len(buf)
        # reszta rezerwuaru
        for row in Xb:
            seen += 1
            if buf.shape[0] < sample_size:
                buf = np.vstack([buf, row[None, :]])
            else:
                j = rng.integers(0, seen)
                if j < sample_size:
                    buf[j] = row
    return _as_float32_c(buf)

def _kmeanspp_on_sample(S, k, rng):
    """k-means++ na próbce S (float32, (m,d)). Zwraca (k,d) float32."""
    m, d = S.shape
    C = np.empty((k, d), dtype=np.float32)
    C[0] = S[rng.integers(0, m)]
    dist2 = np.full(m, np.inf, dtype=np.float32)
    
    for i in range(1, k):
        c = C[i-1]
            # ||x-c||^2 = ||x||^2 + ||c||^2 - 2 x·c
        x2 = np.sum(S*S, axis=1, dtype=np.float32)
        c2 = np.sum(c*c, dtype=np.float32)
        prod = S @ c
        dist2 = np.minimum(dist2, x2 + c2 - 2.0*prod)  # min do najbliższego centrum
        probs = dist2 / (dist2.sum() + 1e-12)
        C[i] = S[rng.choice(m, p=probs)]
    return C

def kmeans_lloyd_streaming(
    X,
    n_clusters: int,
    *,
    max_iter: int = 100,
    n_init: int = 5,
    tol: float = 1e-4,
    chunk_size: int = 100_000,
    init_sample: int = 200_000,
    random_state: int = 42
):
    """
    Dokładny KMeans (Lloyd) liczony strumieniowo po chunkach.
    X: np.ndarray (N,d) lub pd.DataFrame
    Zwraca: (centroids: (k,d) float32, inertia: float)
    """
    # wymiary
    if isinstance(X, pd.DataFrame):
        N, D = X.shape[0], X.shape[1]
    else:
        X = _as_float32_c(X)
        N, D = X.shape

    rng = np.random.default_rng(random_state)
    best_inertia = np.inf
    best_C = None

    # inicjalizacja – próbka rezerwuarowa + k-means++
    S = _reservoir_sample_rows(X, sample_size=min(init_sample, N), chunk_size=chunk_size, rng=rng)

    
    for init_id in range(n_init):
        C = _kmeanspp_on_sample(S, n_clusters, rng)

        for it in range(max_iter):
                # akumulatory (double dla stabilności sum)
            sums = np.zeros_like(C, dtype=np.float64)
            counts = np.zeros(n_clusters, dtype=np.int64)

            C2 = np.sum(C*C, axis=1, dtype=np.float32)  # (k,)

                # --- przypisania na chunkach ---
            for Xb in _iter_batches(X, chunk_size):
                    # Xb: (B,d) float32
                x2 = np.sum(Xb*Xb, axis=1, dtype=np.float32)   # (B,)
                prod = Xb @ C.T                                # (B,k)  BLAS
                dist2 = x2[:, None] + C2[None, :] - 2.0 * prod
                labels = np.argmin(dist2, axis=1)

                    # akumulacja
                for j in range(n_clusters):
                    m = (labels == j)
                    if m.any():
                        counts[j] += int(m.sum())
                        sums[j] += Xb[m].sum(axis=0, dtype=np.float64)

                # aktualizacja centrów
            newC = C.copy()
            nz = counts > 0
            newC[nz] = (sums[nz] / counts[nz][:, None]).astype(np.float32)

                # puste klastry → losowo z próbki (stabilne)
            if (~nz).any():
                repl = _kmeanspp_on_sample(S, (~nz).sum(), rng)
                newC[~nz] = repl

            # kryterium zbieżności (L2 przesunięcia)
            shift = np.linalg.norm((newC - C).astype(np.float64))
            C = newC
            if shift <= tol * max(1.0, np.linalg.norm(C.astype(np.float64))):
                break

            # policz inercję (sumę min dystansów^2) – też strumieniowo
        C2 = np.sum(C*C, axis=1, dtype=np.float32)
        inertia = 0.0
        for Xb in _iter_batches(X, chunk_size):
            x2 = np.sum(Xb*Xb, axis=1, dtype=np.float32)
            prod = Xb @ C.T
            dist2 = x2[:, None] + C2[None, :] - 2.0 * prod
            inertia += float(np.min(dist2, axis=1).sum())

        if inertia < best_inertia:
            best_inertia = inertia
            best_C = C.copy()

    return best_C, best_inertia

def assign_labels_streaming(X, centroids, *, chunk_size=100_000):
    """Przypisz etykiety do wszystkich próbek na bazie centroidów (też strumieniowo)."""
    C = _as_float32_c(centroids)
    C2 = np.sum(C*C, axis=1, dtype=np.float32)
    labels_all = []
    
    for Xb in _iter_batches(X, chunk_size):
        x2 = np.sum(Xb*Xb, axis=1, dtype=np.float32)
        prod = Xb @ C.T
        dist2 = x2[:, None] + C2[None, :] - 2.0 * prod
        labels_all.append(np.argmin(dist2, axis=1))
    return np.concatenate(labels_all, axis=0)


# Załóżmy: X_majority — DataFrame/ndarray tylko z cechami (bez Label), w większości float32
k = 100000                 # realny pułap dla 7.1M; większe wartości rosną bardzo mocno w czasie/RAM
chunk = 100000             # dopasuj do RAM (zmniejsz gdy pamięć ~90%)


X_majority = borderline_df.drop(columns=["Label", "source"])
centroids, inertia = kmeans_lloyd_streaming(
    X_majority,
    n_clusters=k,
    max_iter=30,
    n_init=3,
    tol=1e-4,
    chunk_size=chunk,
    init_sample=200_000,
    random_state=42
)

# Centroidy → DataFrame + metadane, jak u Ciebie
X_cent = pd.DataFrame(centroids, columns=X_majority.columns)
y_cent = pd.Series(np.ones(k, dtype=np.int8), name="Label")
df_majority = pd.concat([X_cent, y_cent], axis=1)
df_majority["source"] = "centroid"

# (opcjonalnie) etykiety dla wszystkich próbek:
# labels = assign_labels_streaming(X_majority, centroids, chunk_size=chunk, blas_threads=threads)


KeyboardInterrupt: 